In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from math import sqrt
import glob

import cv2
import random
from pathlib import Path
import os
from tqdm.notebook import tqdm as tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import shutil
import json

# Подготовка Train

In [2]:
annot_df = pd.read_csv('train_no_heic.csv', sep=';')
annot_df.head()

,image_name,distance
0,img_1596.jpg,4.88
1,img_1600.jpg,1.54
2,img_1601.jpg,3.68
3,img_1603.jpg,2.22
4,img_1605.jpg,3.73


In [3]:
def detections_to_features_df(path_to_labels):
    records = []
    for file_name in os.listdir(path_to_labels):
        with open(path_to_labels+file_name, 'r') as f:
            lines = f.read().splitlines()
            lines = [line[2:] for line in lines]
            x_plate, y_plate, w_plate, h_plate, x_car, y_car, w_car, h_car = list(map(float,(lines[0]+' '+lines[1]).split()))        
            img_name = file_name.split('.')[0]+'.jpg'
            records.append([img_name, x_plate, y_plate, w_plate, h_plate, x_car, y_car, w_car, h_car])
    
    df = pd.DataFrame(records, columns=['image_name', 'x_plate', 'y_plate', 'w_plate',
                                          'h_plate', 'x_car', 'y_car', 'w_car', 'h_car'])
    
    df['plate_area'] = df['w_plate']*df['h_plate']
    df['car_area'] = df['w_car']*df['h_car']
    df['plate_hypotenuse'] = (df['w_plate']**2 + df['h_plate']**2).apply(sqrt)
    df['car_hypotenuse'] = (df['w_car']**2 + df['h_car']**2).apply(sqrt)
    df['car_plate_ratio'] = df['plate_area'] / df['car_area']
    df['plate_proportions'] = df['h_plate']/df['w_plate']
    df['car_proportions'] = df['h_car']/df['w_car']
    df['car_to_bottom'] = 1 - (df['y_car']+df['h_car']/2)
    
    return df

In [4]:
train = detections_to_features_df('train_yolo/labels/')
train = train.merge(annot_df, how='left', on='image_name')
train.shape

(526, 18)

In [5]:
train.head()

,image_name,x_plate,y_plate,w_plate,h_plate,x_car,y_car,w_car,h_car,plate_area,car_area,plate_hypotenuse,car_hypotenuse,car_plate_ratio,plate_proportions,car_proportions,car_to_bottom,distance
0,img_1977.jpg,0.466602,0.626187,0.090690,0.026871,0.464192,0.528925,0.272509,0.280979,0.002437,0.076569,0.094587,0.391421,0.031827,0.296296,1.031083,0.330585,2.20
1,img_1963.jpg,0.460479,0.509495,0.058591,0.014483,0.463112,0.508617,0.171165,0.179504,0.000849,0.030725,0.060355,0.248031,0.027619,0.247191,1.048718,0.401631,3.72
2,img_2503.jpg,0.475444,0.546320,0.041795,0.013519,0.473837,0.501145,0.118212,0.138162,0.000565,0.016332,0.043927,0.181832,0.034596,0.323471,1.168759,0.429774,4.98
3,img_2271.jpg,0.476003,0.590207,0.049012,0.013489,0.474696,0.540411,0.135816,0.157191,0.000661,0.021349,0.050834,0.207738,0.030968,0.275229,1.157380,0.380994,4.30
4,img_2744h.jpg,0.479274,0.508476,0.078934,0.022208,0.478095,0.485804,0.235586,0.234031,0.001753,0.055135,0.081998,0.332071,0.031794,0.281346,0.993399,0.397181,2.66


# Подготовка Test и предсказания алгоритма

## 1. Подготовка test

Несмотря на то, что на большинстве фото распознаны ровно один автомобиль и один номер, на нескольких фото алгоритм детектирования не отработал как предполагалось. Функция analyze_detections ниже решают данную проблему добавлением или удалением детекций.

In [6]:
def analyze_detections(detections):
    '''
    Функция убирает лишние детекции и заполняет пропуски, 
    в случае если распознан только номер/автомобиль или вообще ничего не распознано.
    
    Добавление детекций происходит за счет похожих детекций на других фото
    
    На выходе только две детекции: автомобиль и номер
    '''
    if len(detections)==0:
        # если ничего не распознано берем усредненные детекции по train
        x_car, y_car, w_car, h_car = np.mean(train['x_car']), np.mean(train['y_car']), np.mean(train['w_car']), np.mean(train['h_car'])
        x_plate, y_plate, w_plate, h_plate = np.mean(train['x_plate']), np.mean(train['y_plate']), np.mean(train['w_plate']), np.mean(train['h_plate'])
    elif len(detections)==1:
        label, x, y, w, h, conf = list(map(float,detections[0].split()))
        if int(label)==0:
            x_car, y_car, w_car, h_car = x, y, w, h
            similar_cars = train[abs(train.car_area-w_car*h_car)<0.05]
            Y_OFFSET = np.mean(similar_cars['y_car']-similar_cars['y_plate'])
            WIDTH_RATIO = np.mean(similar_cars['w_car']/similar_cars['w_plate'])
            HEIGHT_RATIO = np.mean(similar_cars['h_car']/similar_cars['h_plate'])
            
            x_plate, y_plate, w_plate, h_plate = x_car, y_car-Y_OFFSET, w_car/WIDTH_RATIO, h_car/HEIGHT_RATIO
        else:
            x_plate, y_plate, w_plate, h_plate = x, y, w, h
            x_car, y_car, w_car, h_car = x_plate+X_OFFSET, y_car+Y_OFFSET, w_plate*WIDTH_RATIO, h_plate*HEIGHT_RATIO
    elif len(detections)>2:
        car_detections = []
        plate_detections = []
        
        for i, line in enumerate(detections):
            label, x, y, w, h, conf = list(map(float,line.split()))
            if int(label)==0:
                car_detections.append([x, y, w, h, conf])
            else:
                plate_detections.append([x, y, w, h, conf])
        
        car_detections, plate_detections = np.array(car_detections), np.array(plate_detections)
        x_car, y_car, w_car, h_car, conf = car_detections[np.argmax(car_detections[:, 4])]
        x_plate, y_plate, w_plate, h_plate, conf = plate_detections[np.argmax(plate_detections[:, 4])]
    
    return x_car, y_car, w_car, h_car, x_plate, y_plate, w_plate, h_plate

def test_detections_to_features_df(path_to_images):
    '''
    Функция принимает на вход путь к папке с детекциями и возвращает датафрейм с фичами по каждой фото
    '''
    records = []
    for img_name in glob.glob(path_to_images+'*.jpg'):
        img_name = img_name.split('/')[-1]
        label_name = img_name.split('.')[0]+'.txt'
        
        if label_name not in os.listdir(path_to_images+'labels/'):
            with open(path_to_images+'labels/'+label_name, 'w') as f:
                f.write('')
        
        with open(path_to_images+'labels/'+label_name, 'r') as f:
            lines = f.read().splitlines()
            if len(lines)!=2:
                x_car, y_car, w_car, h_car, x_plate, y_plate, w_plate, h_plate = analyze_detections(lines)
                records.append([img_name, x_plate, y_plate, w_plate, h_plate, x_car, y_car, w_car, h_car])
            else:
                for i, line in enumerate(lines):
                    label, x, y, w, h, conf = list(map(float,line.split()))
                    if int(label)==0:
                        x_car, y_car, w_car, h_car = x, y, w, h
                    else:
                        x_plate, y_plate, w_plate, h_plate = x, y, w, h

                
                records.append([img_name, x_plate, y_plate, w_plate, h_plate, x_car, y_car, w_car, h_car])
        
    
    df = pd.DataFrame(records, columns=['image_name', 'x_plate', 'y_plate', 'w_plate',
                                          'h_plate', 'x_car', 'y_car', 'w_car', 'h_car'])
    
    df['plate_area'] = df['w_plate']*df['h_plate']
    df['car_area'] = df['w_car']*df['h_car']
    df['plate_hypotenuse'] = (df['w_plate']**2 + df['h_plate']**2).apply(sqrt)
    df['car_hypotenuse'] = (df['w_car']**2 + df['h_car']**2).apply(sqrt)
    df['car_plate_ratio'] = df['plate_area'] / df['car_area']
    df['plate_proportions'] = df['h_plate']/df['w_plate']
    df['car_proportions'] = df['h_car']/df['w_car']
    df['car_to_bottom'] = 1 - (df['y_car']+df['h_car']/2)
    
    return df

In [22]:
test = test_detections_to_features_df('models/yolov5/runs/detect/test_dets_4_conf_65_img_1200/')
test.shape

(521, 17)

In [23]:
test.head()

,image_name,x_plate,y_plate,w_plate,h_plate,x_car,y_car,w_car,h_car,plate_area,car_area,plate_hypotenuse,car_hypotenuse,car_plate_ratio,plate_proportions,car_proportions,car_to_bottom
0,img_2603.jpg,0.462813,0.507083,0.040625,0.010833,0.461563,0.505417,0.124375,0.139167,0.000440,0.017309,0.042045,0.186646,0.025426,0.266666,1.118931,0.424999
1,img_2617.jpg,0.488125,0.518333,0.032500,0.008333,0.487812,0.513750,0.099375,0.117500,0.000271,0.011677,0.033551,0.153888,0.023195,0.256410,1.182390,0.427500
2,img_1875.jpg,0.476250,0.520833,0.032500,0.008333,0.475937,0.505417,0.096875,0.110833,0.000271,0.010737,0.033551,0.147203,0.025224,0.256410,1.144083,0.439167
3,img_1849.jpg,0.473750,0.497083,0.042500,0.012500,0.468750,0.492083,0.128750,0.139167,0.000531,0.017918,0.044300,0.189589,0.029649,0.294118,1.080909,0.438334
4,img_2697_1.jpg,0.487500,0.498750,0.028750,0.009167,0.487188,0.472500,0.088125,0.098333,0.000264,0.008666,0.030176,0.132043,0.030412,0.318841,1.115839,0.478333


## 2. Предсказание дистанции

In [9]:
# Tensorflow sequential model
import tensorflow as tf

model = tf.keras.models.load_model('models/main_model')

In [10]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 1024)              17408     
                                                                 
 dropout_6 (Dropout)         (None, 1024)              0         
                                                                 
 dense_10 (Dense)            (None, 512)               524800    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 1)                 513       
                                                                 
Total params: 542,721
Trainable params: 542,721
Non-trainable params: 0
_________________________________________________________________


In [24]:
test['distance'] = np.squeeze(model.predict(test.loc[:, 'x_plate':'car_to_bottom'].values), axis=1)
test.head()

,image_name,x_plate,y_plate,w_plate,h_plate,x_car,y_car,w_car,h_car,plate_area,car_area,plate_hypotenuse,car_hypotenuse,car_plate_ratio,plate_proportions,car_proportions,car_to_bottom,distance
0,img_2603.jpg,0.462813,0.507083,0.040625,0.010833,0.461563,0.505417,0.124375,0.139167,0.000440,0.017309,0.042045,0.186646,0.025426,0.266666,1.118931,0.424999,5.144441
1,img_2617.jpg,0.488125,0.518333,0.032500,0.008333,0.487812,0.513750,0.099375,0.117500,0.000271,0.011677,0.033551,0.153888,0.023195,0.256410,1.182390,0.427500,6.578923
2,img_1875.jpg,0.476250,0.520833,0.032500,0.008333,0.475937,0.505417,0.096875,0.110833,0.000271,0.010737,0.033551,0.147203,0.025224,0.256410,1.144083,0.439167,6.518903
3,img_1849.jpg,0.473750,0.497083,0.042500,0.012500,0.468750,0.492083,0.128750,0.139167,0.000531,0.017918,0.044300,0.189589,0.029649,0.294118,1.080909,0.438334,4.846159
4,img_2697_1.jpg,0.487500,0.498750,0.028750,0.009167,0.487188,0.472500,0.088125,0.098333,0.000264,0.008666,0.030176,0.132043,0.030412,0.318841,1.115839,0.478333,7.203343


In [25]:
test.shape

(521, 18)

In [26]:
test = test[['image_name', 'distance']]

## 3. Файл submission.csv

Переходим к оригинальным названиям файлов и расширениям

In [27]:
def back_to_heic(img_name):
    '''
    if '_1.jpg' in img_name:
        heic_images.remove(img_name.split('_1.jpg')[0])
        return img_name.split('_1.jpg')[0]+'.heic'
    elif img_name.split('.')[0] in heic_images:
        heic_images.remove(img_name.split('.')[0])
        return img_name.split('.')[0]+'.heic'
    '''
    if '_1.jpg' in img_name:
        result = img_name.split('_1.jpg')[0]+'.heic'
        print(img_name+' converted to ', result)
        return result
    
    if (img_name.split('.')[0] in heic_images) and (img_name.split('.')[0] not in jpg_images):
        result = img_name.split('.')[0]+'.heic'
        print(img_name+' converted to ', result)
        return result

    print(img_name+' - no conversion')
    return img_name


test_annot = pd.read_csv('test.csv', sep=';')
heic_images = []
jpg_images = []
for img_name in test_annot['image_name']:
    if img_name.split('.')[1]=='heic':
        heic_images.append(img_name.split('.')[0])
    else:
        jpg_images.append(img_name.split('.')[0])
        
test['image_name'] = test['image_name'].apply(back_to_heic)

img_2603.jpg - no conversion
img_2617.jpg - no conversion
img_1875.jpg - no conversion
img_1849.jpg - no conversion
img_2697_1.jpg converted to  img_2697.heic
img_2401.jpg - no conversion
img_2367.jpg - no conversion
img_2415.jpg - no conversion
img_1652.jpg - no conversion
img_1646.jpg - no conversion
img_1915.jpg - no conversion
img_2011.jpg - no conversion
img_2777.jpg - no conversion
img_2762.jpg - no conversion
img_2004.jpg - no conversion
img_2776.jpg - no conversion
img_2560.jpg - no conversion
img_1928.jpg - no conversion
img_1914.jpg - no conversion
img_1900.jpg - no conversion
img_2428.jpg - no conversion
img_2414.jpg - no conversion
img_2400.jpg - no conversion
img_2366.jpg - no conversion
img_2399.jpg - no conversion
img_2753_1.jpg converted to  img_2753.heic
img_2616.jpg - no conversion
img_2602.jpg - no conversion
img_2628.jpg - no conversion
img_1876.jpg - no conversion
img_2416.jpg - no conversion
img_2370.jpg - no conversion
img_1679.jpg - no conversion
img_2364.jpg - 

In [28]:
test.head()

,image_name,distance
0,img_2603.jpg,5.144441
1,img_2617.jpg,6.578923
2,img_1875.jpg,6.518903
3,img_1849.jpg,4.846159
4,img_2697.heic,7.203343


In [16]:
test.to_csv('submission_final.csv',index=False, sep=';')